# ARIMA+GARCH Trading Strategy on the S&P500 Stock 

If you've followed our past series on [time series analysis](https://github.com/Auquan/Tutorials#time-series-analysis), you're now all familiar with the powerful ARIMA and GARCH models to model financial series.

In this article, we will apply the previous knowledge in a practical example to develop a trading strategy for the S&P500 

## Strategy Overview

The idea of this strategy is as below:

* For each day, $n$, use differenced logarithmic returns of S&P 500 for the previous $k$ days to fit an optimal ARIMA and GARCH model
* Use the combined model to make a prediction for the next day returns
* If the prediction is positive, go long the stock and if negative, short the stock at $n_{th}$ day's close
* If the prediction is the same as the previous day then do nothing

In this sheet we will work with $k=500$, but this is parameter that can be optimised in order to improve performance or reduce drawdown.

**Note: The backtest is doesnot take comission or slippage into account, hence the performance achieved in a real trading system would be lower than what you see here.**

## Strategy Implementation

In [22]:
import os
import sys

import pandas as pd
import numpy as np

import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
import statsmodels.stats as sms
from arch import arch_model

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

In [23]:
import auquanToolbox.dataloader as dl

end = '2017-01-01'
start = '2014-01-01'
symbols = ['SPX']
data = dl.load_data_nologs('nasdaq', symbols , start, end)['ADJ CLOSE']
# log returns
lrets = 100*np.log(data/data.shift(1)).dropna()

Reading SPX
Data ends before End Date. Filling rest of the data for SPX


In [24]:
def tsplot(y, lags=None, figsize=(15, 10), style='bmh'):
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    with plt.style.context(style):    
        fig = plt.figure(figsize=figsize)
        #mpl.rcParams['font.family'] = 'Ubuntu Mono'
        layout = (3, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))
        qq_ax = plt.subplot2grid(layout, (2, 0))
        pp_ax = plt.subplot2grid(layout, (2, 1))
        
        y.plot(ax=ts_ax)
        ts_ax.set_title('Time Series Analysis Plots')
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.05)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.05)
        sm.qqplot(y, line='s', ax=qq_ax)
        qq_ax.set_title('QQ Plot')        
        scs.probplot(y, sparams=(y.mean(), y.std()), plot=pp_ax)

        plt.tight_layout()
    return

In [25]:
windowLength = 252
foreLength = len(lrets) - windowLength

We also use [previously defined functions](https://github.com/Auquan/Tutorials/blob/master/Time%20Series%20Analysis%20-%204.ipynb) to find best ARIMA model

In [26]:
def _get_best_model(TS):
    best_aic = np.inf 
    best_order = None
    best_mdl = None

    pq_rng = range(1,4) # [1,2,3]
    for i in pq_rng:
            for j in pq_rng:
                try:
                    tmp_mdl = smt.ARIMA(TS, order=(i,0,j)).fit(
                        method='mle', trend='nc'
                    )
                    tmp_aic = tmp_mdl.aic
                    if tmp_aic < best_aic:
                        best_aic = tmp_aic
                        best_order = (i, 0, j)
                        best_mdl = tmp_mdl
                except: continue
    print('aic: %6.5f | order: %s'%(best_aic, best_order))                    
    return best_aic, best_order, best_mdl



At this stage we need to loop through every day in the trading data and fit an appropriate ARIMA and GARCH model to the rolling window of length $k=500$. Given that we try 32 separate ARIMA fits and fit a GARCH model, for each day, the indicator can take a long time to generate.

In [ ]:
signal = 0*lrets[-foreLength:]
for d in range(foreLength):
    # create a rolling window by selecting the values between 1+d and k+d of S&P500 returns
    TS = lrets[(1+d):(windowLength+d)] 
    # Find the best ARIMA fit (we set differencing to 0 since we've already differenced the series once)
    res_tup = _get_best_model(TS)
    order = res_tup[1]
    model = res_tup[2]
    #now that we have our ARIMA fit, we feed this to GARCH model
    p_ = order[0]
    o_ = order[1]
    q_ = order[2]

    am = arch_model(model.resid, p=p_, o=o_, q=q_, dist='StudentsT')
    res = am.fit(update_freq=5, disp='off')
    out = res.forecast(horizon=1, start=None, align='origin')
    signal.iloc[d] = np.sign(out.mean['h.1'].iloc[-1])

aic: 551.01397 | order: (2, 0, 1)
SPX    1.0
Name: 2014-12-31 00:00:00, dtype: float64
aic: 551.64280 | order: (2, 0, 1)
SPX    1.0
Name: 2015-01-02 00:00:00, dtype: float64
aic: 551.63314 | order: (2, 0, 1)
SPX    1.0
Name: 2015-01-05 00:00:00, dtype: float64


C:\Users\Chandini\Miniconda3\envs\auquan\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


aic: 557.53217 | order: (3, 0, 2)
SPX    1.0
Name: 2015-01-06 00:00:00, dtype: float64
aic: 558.25040 | order: (2, 0, 1)
SPX    1.0
Name: 2015-01-07 00:00:00, dtype: float64
aic: 561.43201 | order: (2, 0, 1)
SPX    1.0
Name: 2015-01-08 00:00:00, dtype: float64


C:\Users\Chandini\Miniconda3\envs\auquan\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


aic: 566.89153 | order: (3, 0, 2)
SPX    1.0
Name: 2015-01-09 00:00:00, dtype: float64


...

aic: 658.35335 | order: (3, 0, 2)
SPX    1.0
Name: 2016-10-12 00:00:00, dtype: float64


C:\Users\Chandini\Miniconda3\envs\auquan\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


aic: 658.13801 | order: (3, 0, 2)
SPX    1.0
Name: 2016-10-13 00:00:00, dtype: float64


C:\Users\Chandini\Miniconda3\envs\auquan\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Chandini\Miniconda3\envs\auquan\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Chandini\Miniconda3\envs\auquan\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


aic: 651.33976 | order: (3, 0, 2)
SPX    1.0
Name: 2016-10-14 00:00:00, dtype: float64


## Strategy Results

Now that we have generated our signals, we need to compare its performance to "Buy & Hold".


In [ ]:
rets = pd.DataFrame(index = signal.index, columns=['Buy and Hold', 'Strategy'])
rets['Buy and Hold'] = lrets[-foreLength:]
rets['Strategy'] = signal
rets['Strategy'] = rets['Strategy']*rets['Buy and Hold']
eqCurves['Buy and Hold']=rets['Buy and Hold'].cumsum()+1
eqCurves['Strategy'] = rets['Strategy'].cumsum()+1
eqCurves['Strategy'].plot(figsize=(10,8))
eqCurves['Buy and Hold'].plot()
plt.legend()
plt.show()